In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [3]:
df=pd.read_csv("/kaggle/input/heart-disease-prediction/Heart_Disease_Prediction.csv")
df.head(5)

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      270 non-null    int64  
 1   Sex                      270 non-null    int64  
 2   Chest pain type          270 non-null    int64  
 3   BP                       270 non-null    int64  
 4   Cholesterol              270 non-null    int64  
 5   FBS over 120             270 non-null    int64  
 6   EKG results              270 non-null    int64  
 7   Max HR                   270 non-null    int64  
 8   Exercise angina          270 non-null    int64  
 9   ST depression            270 non-null    float64
 10  Slope of ST              270 non-null    int64  
 11  Number of vessels fluro  270 non-null    int64  
 12  Thallium                 270 non-null    int64  
 13  Heart Disease            270 non-null    object 
dtypes: float64(1), int64(12), 

In [5]:
df.shape

(270, 14)

In [6]:
columns=df.columns
print(columns)

Index(['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120',
       'EKG results', 'Max HR', 'Exercise angina', 'ST depression',
       'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease'],
      dtype='object')


In [7]:
category_col= [col for col in df.columns if df[col].nunique() < 10]
category_col

['Sex',
 'Chest pain type',
 'FBS over 120',
 'EKG results',
 'Exercise angina',
 'Slope of ST',
 'Number of vessels fluro',
 'Thallium',
 'Heart Disease']

In [8]:
for col in category_col:
    print(df[col].value_counts())

Sex
1    183
0     87
Name: count, dtype: int64
Chest pain type
4    129
3     79
2     42
1     20
Name: count, dtype: int64
FBS over 120
0    230
1     40
Name: count, dtype: int64
EKG results
2    137
0    131
1      2
Name: count, dtype: int64
Exercise angina
0    181
1     89
Name: count, dtype: int64
Slope of ST
1    130
2    122
3     18
Name: count, dtype: int64
Number of vessels fluro
0    160
1     58
2     33
3     19
Name: count, dtype: int64
Thallium
3    152
7    104
6     14
Name: count, dtype: int64
Heart Disease
Absence     150
Presence    120
Name: count, dtype: int64


In [9]:
df.drop(['Sex','Exercise angina','Slope of ST','Number of vessels fluro','Thallium','Max HR'],inplace=True,axis=1)

In [10]:
df

,Age,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,ST depression,Heart Disease
0,70,4,130,322,0,2,2.4,Presence
1,67,3,115,564,0,2,1.6,Absence
2,57,2,124,261,0,0,0.3,Presence
3,64,4,128,263,0,0,0.2,Absence
4,74,2,120,269,0,2,0.2,Absence
...,...,...,...,...,...,...,...,...
265,52,3,172,199,1,0,0.5,Absence
266,44,2,120,263,0,0,0.0,Absence
267,56,2,140,294,0,2,1.3,Absence
268,57,4,140,192,0,0,0.4,Absence


In [11]:
mapping = {'Presence': 1, 'Absence': 0}
df['Heart Disease'] = df['Heart Disease'].map(mapping)

In [12]:
x=df.drop('Heart Disease', axis=1)
y=df['Heart Disease']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

accuracy, conf_matrix

(0.6851851851851852,
 array([[25,  8],
        [ 9, 12]]))

In [15]:
print(class_report)

              precision    recall  f1-score   support

           0       0.74      0.76      0.75        33
           1       0.60      0.57      0.59        21

    accuracy                           0.69        54
   macro avg       0.67      0.66      0.67        54
weighted avg       0.68      0.69      0.68        54



In [16]:
from sklearn.ensemble import RandomForestClassifier 

rfc = RandomForestClassifier() 
rfc.fit(X_train, y_train) 

yPred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, yPred)
conf_matrix = confusion_matrix(y_test, yPred)
accuracy,conf_matrix

(0.6481481481481481,
 array([[25,  8],
        [11, 10]]))

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, and y_test are your training and testing data
# Initialize SVM classifier
svm_model = SVC()

# Train the SVM model
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_svm = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_svm)
print('Accuracy of the SVM model is =', accuracy)
conf_matrix = confusion_matrix(y_test, yPred)
print(conf_matrix)

Accuracy of the SVM model is = 0.6666666666666666
[[25  8]
 [11 10]]


In [18]:
import pickle
pickle.dump(model,open('/kaggle/working/model.pkl','wb'))

In [19]:
df.to_csv('/kaggle/working/data.csv', index=False)